In [1]:
import pandas as pd
import schedule
import time
from logger import logging

import krakenex
from pykrakenapi import KrakenAPI
from talib import MACD
api = krakenex.API()
k = KrakenAPI(api)
api.load_key('kraken_keys.py')

In [2]:
now = str(pd.Timestamp.today())[0:16]
logging.info('Calculating metrics for {now}...'.format(now=now))

[10/Nov/2020 20:33:28] INFO - Calculating metrics for 2020-11-10 20:33...


In [62]:
def Four_MA(close):
    
    ma_8 = close.rolling(8).mean()
    ma_13 = close.rolling(13).mean()
    ma_21 = close.rolling(21).mean()
    ma_55 = close.rolling(55).mean()
    return ma_8, ma_13, ma_21, ma_55

# Calculate metrics
currency = 'ZUSD'
crypto = 'XETH'
pair = crypto + currency
df = k.get_ohlc_data(pair, interval=240, ascending=True)[0]
df.index = df.index.tz_localize(tz='UTC').tz_convert('US/Central')
ma_8, ma_13, ma_21, ma_55 = Four_MA(df['close'])
logging.info(f'MA are (8, 13, 21, 55): {ma_8[-1]:.2f} | {ma_13[-1]:.2f} | {ma_21[-1]:.2f} | {ma_55[-1]:.2f}')

[10/Nov/2020 20:57:45] INFO - MA are (8, 13, 21, 55): 449.68 | 449.16 | 447.04 | 423.16


In [63]:
volume = k.get_account_balance()
cash_on_hand = volume.loc[currency][0]
current_price = df['close'][-1]
shares = cash_on_hand/current_price

In [64]:
crypto_on_hand = volume.loc[crypto][0]

In [65]:
crypto_on_hand = volume.loc[crypto][0]
open_position = [True if len(k.get_open_positions()) > 0 else False][0]
otype = 'buy'

In [66]:
api.query_private('AddOrder', {'pair': pair, 
                               'type': otype, 
                               'ordertype':'limit', 
                               'price': current_price, 
                               'volume': shares,
                               'expiretm': '+30',
                               'close[ordertype]': 'stop-loss', 
                               'close[price]': '-1%'})

{'error': [],
 'result': {'descr': {'order': 'buy 0.67387923 ETHUSD @ limit 463.03',
   'close': 'close position @ stop loss -1.0000%'},
  'txid': ['OIJMQN-YKVIR-23TO45']}}

In [54]:
api.query_private('AddOrder', {'pair': pair, 
                               'type': 'sell', 
                               'ordertype':'stop-loss', 
                               'price': '460', 
                               'volume': 0.67})

{'error': [],
 'result': {'descr': {'order': 'sell 0.67000000 ETHUSD @ stop loss 460.00'},
  'txid': ['O33O3B-NBMNH-HOFKFN']}}

In [31]:
k.add_standard_order(pair=pair, type=otype, ordertype='limit', volume=shares, price=current_price, expiretm='+30', close_ordertype='stop-loss', close_price='-1%')

{'descr': {'order': 'buy 0.67740726 ETHUSD @ limit 462.06',
  'close': 'close position @ stop loss -1.0000%'}}

In [68]:
k.get_open_orders(trades=True).T

""


In [74]:
k.get_

(                                         ledger_id                  refid  \
 dtime                                                                       
 2020-11-11 02:57:47.209500074  LOUKFC-YED42-X4XULW    TH4SXP-HKUYG-SYIGUI   
 2020-11-11 02:57:47.209000111  LWSYVJ-YMLMG-GDDQAY    TH4SXP-HKUYG-SYIGUI   
 2020-11-11 02:57:26.484100103  LEKBXE-SP6SD-UQF7K3    TTDLAN-RCXCX-UXBVOY   
 2020-11-11 02:57:26.483700037  LXSC4G-A4RGA-M2M4DA    TTDLAN-RCXCX-UXBVOY   
 2020-11-11 02:49:23.190200090  LXYNQJ-XUODP-HUEUOF    TIRUGZ-YCHNS-DLXQIL   
 2020-11-11 02:49:23.189699888  LYRGDW-DOHIE-OMO7WV    TIRUGZ-YCHNS-DLXQIL   
 2020-11-11 02:36:47.438699961  LS3AHF-GY5EU-7SRLTL    T6PSVN-SEMKY-AJHYT2   
 2020-11-11 02:36:47.438100100  LLGFDM-WLW4S-B5BXHZ    T6PSVN-SEMKY-AJHYT2   
 2020-11-11 02:33:57.141599894  LPZDCQ-KKYJZ-MOH4MB    TYHQXS-X6KAP-I6EEO4   
 2020-11-11 02:33:57.141000032  LLTY5L-WFYGD-C3TXFB    TYHQXS-X6KAP-I6EEO4   
 2020-10-08 13:56:12.971400023  L6PDGQ-RZOK7-RE7E3J    TITSYP-G2

In [12]:
help(KrakenAPI)

Help on class KrakenAPI in module pykrakenapi.pykrakenapi:

class KrakenAPI(builtins.object)
 |  KrakenAPI(api, tier='Intermediate', retry=1, crl_sleep=5)
 |  
 |  A python implementation of the Kraken API.
 |  
 |  Implements the Kraken API methods using the low-level krakenex python
 |  package. See
 |  https://www.kraken.com/help/api
 |  and
 |  https://github.com/veox/python3-krakenex
 |  
 |  Parameters
 |  ----------
 |  api : krakenex.API
 |      An instance of the krakenex.API class. A reference to the input
 |      is created and accessible via ``KrakenAPI.api``.
 |  
 |  tier : str, optional (default='Intermediate')
 |      Your Kraken tier level, used to adjust the limit of the call rate to
 |      the Kraken API in order to prevent 15 minute temporary lockouts.
 |      Must be one of {'None', 'Starter', 'Intermediate', 'Pro'}.
 |      Set tier='None' to disable the call rate limiter.
 |      See https://support.kraken.com/hc/en-us/articles/206548367.
 |  
 |  retry : float,